In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import pyodbc
from googletrans import Translator

# DB 연결 세팅 - 민감 정보 직접 입력 필요
conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=여기에_서버주소_입력;'
    'DATABASE=여기에_데이터베이스명_입력;'
    'UID=여기에_아이디_입력;'
    'PWD=여기에_비밀번호_입력'
)
cursor = conn.cursor()

translator = Translator()

# 고정 단어 번역 방지용 사전 (필요시 수정)
fixed_terms = {
    "백반": "A meal with rice",
    "비빔밥": "Bibimbap",
    "돈가스김치나베": "Pork cutlet kimchi hot pot",
    "단무지": "Pickled radish",
    "새우튀김우동": "Shrimp tempura udon",
    "장국": "Soup base",
    "우동장국": "Udon soup base",
    "사각꼬지어묵국": "Square fish cake soup",
    "참함박스테이크": "Hamburger steak",
    "페퍼로니디럭스피자": "Pepperoni deluxe pizza"
}

def translate_text(text, src='ko', dest='en'):
    try:
        for term, fixed_translation in fixed_terms.items():
            if term in text:
                text = text.replace(term, fixed_translation)
        translated = translator.translate(text, src=src, dest=dest)
        return translated.text
    except Exception as e:
        print(f"번역 오류: {e} / 텍스트: {text}")
        return text

### 1. menu_data 크롤링 및 DB 업데이트

def update_menu_data():
    url_menu = '여기에_첫번째_메뉴_데이터_URL_입력'  # 첫 번째 메뉴 크롤링 URL 직접 입력 필요
    res = requests.get(url_menu)
    if res.status_code != 200:
        print("menu_data 페이지 요청 실패")
        return

    soup = BeautifulSoup(res.text, 'html.parser')
    # TODO: menu_data 크롤링 및 DataFrame 생성 로직 작성
    # 예: dates, categories, meal_times, menus 추출해서 df_menu 생성

    # --- 예시 ---
    # df_menu = pd.DataFrame(columns=['menu_date','menu_type','meal_time','menu_items'])
    # ... 실제 크롤링 후 데이터프레임에 채우기 ...

    cursor.execute("TRUNCATE TABLE menu_data")
    conn.commit()

    for _, row in df_menu.iterrows():
        cursor.execute(
            "INSERT INTO menu_data (menu_date, menu_type, meal_time, menu_items) VALUES (?, ?, ?, ?)",
            row['menu_date'], row['menu_type'], row['meal_time'], row['menu_items']
        )
    conn.commit()

    cursor.execute("SELECT menu_id, menu_items FROM menu_data WHERE menu_items IS NOT NULL")
    rows = cursor.fetchall()
    for menu_id, menu_items in rows:
        menu_en = translate_text(menu_items, dest='en')
        menu_zh = translate_text(menu_items, dest='zh-cn')
        cursor.execute("UPDATE menu_data SET menu_en=?, menu_zh=? WHERE menu_id=?", menu_en, menu_zh, menu_id)
    conn.commit()
    print("menu_data 업데이트 완료")

### 2. dormitory_menu 크롤링 및 DB 업데이트

def update_dormitory_menu():
    url_dorm = '여기에_기숙사메뉴_URL_입력'  # 두 번째 기숙사 메뉴 URL 직접 입력 필요
    res = requests.get(url_dorm)
    if res.status_code != 200:
        print("dormitory_menu 페이지 요청 실패")
        return

    soup = BeautifulSoup(res.text, 'html.parser')
    # TODO: dormitory_menu 크롤링 및 DataFrame 생성 로직 작성

    # --- 예시 ---
    # df_dorm = pd.DataFrame(columns=['menu_date','menu_type','meal_time','menu_items'])
    # ... 실제 크롤링 후 데이터프레임에 채우기 ...

    cursor.execute("TRUNCATE TABLE dormitory_menu")
    conn.commit()

    for _, row in df_dorm.iterrows():
        cursor.execute(
            "INSERT INTO dormitory_menu (menu_date, menu_type, meal_time, menu_items) VALUES (?, ?, ?, ?)",
            row['menu_date'], row['menu_type'], row['meal_time'], row['menu_items']
        )
    conn.commit()

    cursor.execute("SELECT menu_id, menu_items FROM dormitory_menu WHERE menu_items IS NOT NULL")
    rows = cursor.fetchall()
    for menu_id, menu_items in rows:
        menu_en = translate_text(menu_items, dest='en')
        menu_zh = translate_text(menu_items, dest='zh-cn')
        cursor.execute("UPDATE dormitory_menu SET menu_en=?, menu_zh=? WHERE menu_id=?", menu_en, menu_zh, menu_id)
    conn.commit()
    print("dormitory_menu 업데이트 완료")

### 3. notices 크롤링 및 DB 업데이트

def update_notices():
    url_notices = '여기에_세번째_공지사항_URL_입력'  # 공지사항 페이지 URL 직접 입력 필요
    res = requests.get(url_notices)
    if res.status_code != 200:
        print("notices 페이지 요청 실패")
        return

    soup = BeautifulSoup(res.text, 'html.parser')
    # TODO: notices 크롤링 및 DataFrame 생성 로직 작성

    # --- 예시 ---
    # df_notice = pd.DataFrame(columns=['notice_id','notice_title','notice_content'])
    # ... 크롤링 후 데이터프레임 생성 ...

    # 필요시 기존 공지사항 초기화
    # cursor.execute("TRUNCATE TABLE notices")
    # conn.commit()

    for _, row in df_notice.iterrows():
        cursor.execute(
            "INSERT INTO notices (notice_id, notice_title, notice_content) VALUES (?, ?, ?)",
            row['notice_id'], row['notice_title'], row['notice_content']
        )
    conn.commit()

    cursor.execute("SELECT notice_id, notice_title, notice_content FROM notices")
    notices = cursor.fetchall()
    for notice_id, title, content in notices:
        title_en = translate_text(title, dest='en')
        title_zh = translate_text(title, dest='zh-cn')
        content_en = translate_text(content, dest='en')
        content_zh = translate_text(content, dest='zh-cn')
        cursor.execute(
            "UPDATE notices SET notice_title_en=?, notice_title_zh=?, notice_content_en=?, notice_content_zh=? WHERE notice_id=?",
            title_en, title_zh, content_en, content_zh, notice_id
        )
    conn.commit()
    print("notices 업데이트 완료")

### 메인 실행부

if __name__ == "__main__":
    update_menu_data()
    update_dormitory_menu()
    update_notices()

    cursor.close()
    conn.close()
